# Batch Run + Evaluation for Ticket Categorization

This notebook:
1. Loads your taxonomy RAG pipeline from `graph_agent_taxonomy_rag.py` (from the canvas).
2. Runs it over `dataset_for_categorization.csv` with columns **Case ID, ticket, employee**.
3. Saves predictions to `predictions_with_pipeline.csv`.
4. Evaluates against the **noisy** `employee` column by detecting which tier (Domain/Cat1/Cat2/Cat3) the employee label belongs to and comparing at that tier.
5. Prints summary metrics and a confusion matrix for the top Cat3 classes.

> Note: `employee` labels are ~50–70% accurate and may be inconsistent. Treat these metrics as **agreement with the existing process**, not absolute ground truth.

## 0) (Optional) Install dependencies

In [1]:
# If you're in a clean environment, uncomment:
# !pip install -q langgraph langchain langchain-google-genai langchain-community faiss-cpu rank_bm25 pandas python-dotenv langfuse scikit-learn tqdm

## 1) Imports & configuration

In [ ]:
import os

os.environ.setdefault("GRPC_VERBOSITY", "NONE")
os.environ.setdefault("GLOG_minloglevel", "2")
import json
import time
from pathlib import Path

import pandas as pd
from tqdm import tqdm

# For evaluation
import numpy as np
from collections import Counter
from sklearn.metrics import confusion_matrix

# Import your pipeline (make sure the file exists locally)
from src.seq_all_rag import run_pipeline

# ---- Paths ----
DATASET_CSV = "data/evaluation_dataset2.csv"  # input with Case ID, ticket, employee
TAXONOMY_CSV = "data/cleaned.csv"  # your taxonomy file
OUTPUT_PRED_CSV = "data/predictions_with_pipeline.csv"

# ---- Controls ----
TICKET_MAX_CHARS = 2000  # truncate very long tickets (head+tail kept below)
HEAD_KEEP = 1400
TAIL_KEEP = 600
SLEEP_BETWEEN = 0.0  # set to small number if you want to throttle requests

# Optional: set your env vars for Google + Langfuse before running (or via .env file)
# os.environ["GOOGLE_API_KEY"] = "...."
# os.environ["LANGFUSE_PUBLIC_KEY"] = "...."
# os.environ["LANGFUSE_SECRET_KEY"] = "...."
# os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com"

## 2) Helpers (truncation, normalization, tier detection)

In [3]:
def truncate_ticket(
    text: str, max_chars=TICKET_MAX_CHARS, head_keep=HEAD_KEEP, tail_keep=TAIL_KEEP
) -> str:
    if not isinstance(text, str):
        return ""
    text = text.strip()
    if len(text) <= max_chars:
        return text
    head = text[:head_keep]
    tail = text[-tail_keep:]
    return head + "\n...\n" + tail


def normalize_label(s):
    if not isinstance(s, str):
        return ""
    return " ".join(s.strip().split())  # collapse multiple spaces


# Load taxonomy and build lookup sets to detect which tier a label belongs to
tax = pd.read_csv(TAXONOMY_CSV).fillna("")
for col in ["Domain", "Cat1", "Cat2", "Cat3"]:
    if col in tax.columns:
        tax[col] = tax[col].astype(str).str.strip()

DOMAINS = set(tax["Domain"].unique().tolist())
CAT1S = set(tax["Cat1"].unique().tolist())
CAT2S = set(tax["Cat2"].unique().tolist())
CAT3S = set(tax["Cat3"].unique().tolist())


def detect_employee_tier(emp_label: str):
    lab = normalize_label(emp_label)
    if lab in CAT3S:
        return "Cat3"
    if lab in CAT2S:
        return "Cat2"
    if lab in CAT1S:
        return "Cat1"
    if lab in DOMAINS:
        return "Domain"
    return None


# Optional: for confusion matrices, get the full list of Cat3 labels
ALL_CAT3 = sorted(list(CAT3S))
cat3_to_idx = {c: i for i, c in enumerate(ALL_CAT3)}

## 3) Batch run pipeline over dataset

In [4]:
df = pd.read_csv(DATASET_CSV).fillna("")
assert set(["case_id", "case", "ajiltan"]).issubset(
    df.columns
), "Input CSV must have columns: case_id, case, ajiltan"

pred_rows = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    case_id = row["case_id"]
    ticket_text = truncate_ticket(row["case"])
    try:
        out = run_pipeline(ticket_text, TAXONOMY_CSV)
        pred_rows.append(
            {
                "case_id": case_id,  # <-- consistent key
                "pred_domain": out.get("domain", ""),
                "pred_cat1": out.get("category_1", ""),
                "pred_cat2": out.get("category_2", ""),
                "pred_cat3": out.get("category_3", ""),
            }
        )
    except Exception as e:
        pred_rows.append(
            {
                "case_id": case_id,  # <-- consistent key
                "pred_domain": "",
                "pred_cat1": "",
                "pred_cat2": "",
                "pred_cat3": "",
                "error": str(e),
            }
        )
    if SLEEP_BETWEEN > 0:
        time.sleep(SLEEP_BETWEEN)

pred_df = pd.DataFrame(pred_rows)
out_df = df.merge(pred_df, on="case_id", how="left")
out_df.to_csv(OUTPUT_PRED_CSV, index=False)
out_df.head()

  0%|          | 0/21 [00:00<?, ?it/s]WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
E0000 00:00:1760338734.427080  890740 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1760338734.430147  890740 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1760338735.613568  890740 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1760338736.437708  890740 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1760338736.438896  890740 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1760338737.656805  890740 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1760338738.444014  890740 alts_credentials.cc:9

,case_id,case,ajiltan,pred_domain,pred_cat1,pred_cat2,pred_cat3
0,145075,"Gantuya': ""Туслах цэс""\n'Unitel': ""U-Bot-с хэр...",Mobile > Үндсэн үйлчилгээ > Багц солихтой холб...,Mobile,Дата,Дата багцуудтай холбоотой,L2- Дата багцын үнийн дүн зөрүүтэй харагдсанаа...
1,145050,"Dx Bataa Baynmynkh Bataa': ""Sain bnuu""\n'Unite...",Mobile > Дугаар > Дагалдах эрхтэй холбоотой,Mobile,Дугаар,"Дугаар хааж, нээлгэхтэй холбоотой",P-Дугаар нээлгэх хүсэлт
2,145034,"Altbazar Altansarnai': ""Үглээ юнитэл""\n'Unitel...",Mobile > Үндсэн үйлчилгээ > Гар утасны тохирго...,Mobile,Үндсэн үйлчилгээ,Гар утасны тохиргоотой холбоотой,
3,144958,"Б. Нансалмаа': ""Туслах цэс""\n'Unitel': ""U-Bot-...",Mobile > Дугаар > Пин пук кодтой холбоотой,Mobile,Дугаар,Сим сэргээхтэй холбоотой,L2-Unitel app сим сэргээхэд амжилтгүй алдаа за...
4,144952,"Н. Энжи': ""Get started""\n'Н. Энжи': ""Ажилтанта...",Mobile > Үндсэн үйлчилгээ > Гар утас лизингтэй...,Mobile,Бусад,Digital сувагтай холбоотой,L2-Лизингийн хугацаа дууссан ч багц хооронд ши...


## 4) Evaluation (agreement vs. employee labels)

In [5]:
out_df = pd.read_csv(OUTPUT_PRED_CSV).fillna("")
out_df["employee_norm"] = out_df["ajiltan"].apply(normalize_label)
out_df["emp_tier"] = out_df["employee_norm"].apply(detect_employee_tier)


def compare_at_tier(row):
    tier = row["emp_tier"]
    emp = row["employee_norm"]
    if tier == "Domain":
        return emp == normalize_label(row.get("pred_domain", ""))
    if tier == "Cat1":
        return emp == normalize_label(row.get("pred_cat1", ""))
    if tier == "Cat2":
        return emp == normalize_label(row.get("pred_cat2", ""))
    if tier == "Cat3":
        return emp == normalize_label(row.get("pred_cat3", ""))
    return False


out_df["match_at_emp_tier"] = out_df.apply(compare_at_tier, axis=1)

summary = (
    out_df.groupby("emp_tier")["match_at_emp_tier"]
    .agg(["count", "mean"])
    .rename(columns={"mean": "accuracy"})
)
summary.loc["ALL"] = [len(out_df), out_df["match_at_emp_tier"].mean()]
summary

,count,accuracy
emp_tier,,
ALL,21.0,0.0


### 4.1 Breakdown by Domain (using predicted domain)

In [6]:
by_domain = (
    out_df.groupby("pred_domain")["match_at_emp_tier"]
    .agg(["count", "mean"])
    .rename(columns={"mean": "agreement"})
    .sort_values("count", ascending=False)
)
by_domain.head(20)

,count,agreement
pred_domain,,
Mobile,18,0.0
Toki,2,0.0
IPTV,1,0.0


### 4.2 Confusion matrix for Cat3 (top classes only)

In [7]:
cat3_rows = out_df[out_df["emp_tier"] == "Cat3"].copy()
TOP_N = 40
top_emp_cat3 = cat3_rows["employee_norm"].value_counts().head(TOP_N).index.tolist()
cm_df = cat3_rows[cat3_rows["employee_norm"].isin(top_emp_cat3)].copy()

from sklearn.metrics import confusion_matrix

y_true = [normalize_label(x) for x in cm_df["employee_norm"].tolist()]
y_pred = [normalize_label(x) for x in cm_df["pred_cat3"].tolist()]

labels = sorted(list(set(top_emp_cat3 + list(set(y_pred)))))
cm = confusion_matrix(y_true, y_pred, labels=labels)
cm_df_out = pd.DataFrame(
    cm, index=[f"T:{l}" for l in labels], columns=[f"P:{l}" for l in labels]
)
cm_df_out.head(20)

ValueError: 'labels' should contains at least one label.

In [8]:
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd


def safe_normalize(x):
    s = normalize_label(x)
    return "∅" if s == "" else s


def cat3_confusion_and_report(out_df, top_n=40):
    df = out_df.copy().fillna("")
    if "emp_tier" not in df.columns:
        df["employee_norm"] = df["ajiltan"].apply(normalize_label)
        df["emp_tier"] = df["employee_norm"].apply(detect_employee_tier)

    cat3_rows = df[df["emp_tier"] == "Cat3"].copy()
    if cat3_rows.empty:
        print("No rows where employee label is Cat3 — skipping confusion matrix.")
        return (None, None, None)

    cat3_rows["y_true"] = cat3_rows["employee_norm"].apply(safe_normalize)
    cat3_rows["y_pred"] = cat3_rows["pred_cat3"].apply(safe_normalize)

    # Apply top-N; if it drops everything, fall back to ALL
    if top_n is not None:
        true_counts = cat3_rows["y_true"].value_counts()
        keep = set(true_counts.head(top_n).index.tolist())
        cm_df = cat3_rows[cat3_rows["y_true"].isin(keep)].copy()
        if cm_df.empty:  # fallback
            cm_df = cat3_rows
    else:
        cm_df = cat3_rows

    y_true = cm_df["y_true"].tolist()
    y_pred = cm_df["y_pred"].tolist()
    labels = sorted(set(y_true) | set(y_pred))

    if len(labels) == 0:
        print("Zero labels to evaluate even after fallback.")
        return (None, None, None)

    cm = confusion_matrix(y_true, y_pred, labels=labels)
    cm_df_out = pd.DataFrame(
        cm, index=[f"T:{l}" for l in labels], columns=[f"P:{l}" for l in labels]
    )

    print(f"Rows evaluated: {len(cm_df)}  |  Unique labels: {len(labels)}")
    print("\nClassification report (micro/macro averages at bottom):")
    print(classification_report(y_true, y_pred, labels=labels, zero_division=0))

    display(cm_df_out.head(20))
    return (cm_df_out, labels, cm_df)


# Use it:
cm_df_out, labels_used, rows_used = cat3_confusion_and_report(out_df, top_n=40)

No rows where employee label is Cat3 — skipping confusion matrix.


### 4.3 Save compact metrics JSON

In [9]:
metrics = {
    "overall_agreement_at_emp_tier": float(out_df["match_at_emp_tier"].mean()),
    "counts_by_emp_tier": out_df["emp_tier"].value_counts(dropna=False).to_dict(),
    "domain_breakdown": out_df.groupby("pred_domain")["match_at_emp_tier"]
    .mean()
    .sort_values(ascending=False)
    .to_dict(),
}
with open("metrics_summary.json", "w", encoding="utf-8") as f:
    json.dump(metrics, f, ensure_ascii=False, indent=2)
metrics

{'overall_agreement_at_emp_tier': 0.0,
 'counts_by_emp_tier': {None: 21},
 'domain_breakdown': {'IPTV': 0.0, 'Mobile': 0.0, 'Toki': 0.0}}

## 5) Optional: spot-check disagreements

In [10]:
sample = out_df[out_df["match_at_emp_tier"] == False]
if len(sample) > 0:
    sample = sample.sample(min(10, len(sample)), random_state=42)
sample[
    [
        "case_id",
        "case",
        "ajiltan",
        "emp_tier",
        "pred_domain",
        "pred_cat1",
        "pred_cat2",
        "pred_cat3",
    ]
]

,case_id,case,ajiltan,emp_tier,pred_domain,pred_cat1,pred_cat2,pred_cat3
0,145075,"Gantuya': ""Туслах цэс""\n'Unitel': ""U-Bot-с хэр...",Mobile > Үндсэн үйлчилгээ > Багц солихтой холб...,None,Mobile,Дата,Дата багцуудтай холбоотой,L2- Дата багцын үнийн дүн зөрүүтэй харагдсанаа...
17,142619,"Mishka Naraa': ""Ажилтантай холбогдох""\n'tulga....","Mobile > Төлбөр > Төлөлт, гүйлгээтэй холбоотой",None,Mobile,Дугаар,"Дугаар хааж, нээлгэхтэй холбоотой",L2- Устах ёстой дугаар хугацаандаа устаагүйгээ...
15,144724,"Bayar BT': ""хүрд яаж эргүүлэх вэ""\n'Unitel': ""...",Mobile > Үндсэн үйлчилгээ > Урамшуулалтай холб...,None,Toki,Toki Mobile,Үндсэн нөхцөл,
1,145050,"Dx Bataa Baynmynkh Bataa': ""Sain bnuu""\n'Unite...",Mobile > Дугаар > Дагалдах эрхтэй холбоотой,None,Mobile,Дугаар,"Дугаар хааж, нээлгэхтэй холбоотой",P-Дугаар нээлгэх хүсэлт
8,144757,"Panchi Panchi': ""Get started""\n'Unitel': ""Сайн...",Mobile > Үндсэн үйлчилгээ > Нэр шилжүүлэхтэй х...,None,Mobile,Нэмэлт үйлчилгээ,Unitel.mn тодорхойлолт авах,
5,144951,"Huslee Huslen': ""5k ymu 10k aar tseneglevel ur...",Mobile > Нэгж > Цэнэглэгч карттай холбоотой,None,Mobile,Нэгж,"Цэнэглэлт, гүйлгээтэй холбоотой",L2-Unitel app-р авсан нэгж ороогүйгээс шалтгаа...
11,144970,"Ану Ану': ""Get Started""\n'Ану Ану': ""Ажилтанта...",Mobile > Дугаар > Шинэ хэрэглэгчтэй холбоотой,None,Mobile,Дугаар,Пин пук кодтой холбоотой,L1 - Пук код авах хүсэлт
3,144958,"Б. Нансалмаа': ""Туслах цэс""\n'Unitel': ""U-Bot-...",Mobile > Дугаар > Пин пук кодтой холбоотой,None,Mobile,Дугаар,Сим сэргээхтэй холбоотой,L2-Unitel app сим сэргээхэд амжилтгүй алдаа за...
18,142568,"Khishig Dorj': ""Get started""\n'Khishig Dorj': ...",Mobile > Дугаар > Шинэ хэрэглэгчтэй холбоотой,None,Mobile,Дугаар,Шинэ хэрэглэгчтэй холбоотой,L2-Unitel app-с шинэ дугаар авахад алдаа заасн...
16,142692,"Any Any': ""Get started""\n'Any Any': ""Ажилтанта...",Mobile > Дата > Зарцуулалттай холбоотой,None,Mobile,Дата,Зарцуулалттай холбоотой,L2-Хэрэглэгч хэрэглээгээ хүлээн зөвшөөрөөгүйгэ...
